In [13]:
#default_exp qdrant

In [14]:
#export
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from tqdm import tqdm

from qdrant_client import QdrantClient
import qdrant_client
collection_name = "food_images"


##### production
cd
docker run -p 6333:6333 \
    -v $(pwd)/qdrant_storage_prod:/qdrant/storage \
    --name qdrant_prod \
    qdrant/qdrant
docker update --restart unless-stopped qdrant_prod

In [9]:
# !pip install --upgrade qdrant_client

In [10]:
#export
dim = 768 #+ onehot.n_dim()
QdrantClient.get_collections = lambda self: [c['name'] for c in self.http.collections_api.get_collections().dict()['result']['collections']]
QdrantClient.collection_len = lambda self, name: self.http.collections_api.get_collection(name).dict()['result']['vectors_count']

In [11]:
#export
prod_client = QdrantClient(host='localhost', port=6333)
dev_client  = QdrantClient(host='localhost', port=6334)


In [12]:
client = prod_client

In [13]:
collection_name

'food_images'

In [14]:
client

In [15]:
client.recreate_collection(collection_name,dim,qdrant_client.http.models.Distance.DOT)

{
  "vector_size": 768,
  "distance": "Dot",
  "shard_number": null,
  "on_disk_payload": null,
  "hnsw_config": null,
  "wal_config": null,
  "optimizers_config": null
}


In [16]:
client.get_collections()

['food_images']

In [20]:
client.collection_len(collection_name)

0

In [18]:
!nbdev_build_lib

Converted 00_clipmodel.ipynb.
Converted 00_custom_pandas.ipynb.
Converted 00_paths.ipynb.
Converted 00_progress_check.ipynb.
Converted 00_psql.ipynb.
Converted 00_qdrant.ipynb.
Converted 00_tools.ipynb.
Converted 01_multiple_foods.ipynb.
Converted 01_multiple_foods_segmantation.ipynb.
Converted 01_search.ipynb.
Converted 0_template copy 2.ipynb.
Converted 0_template copy.ipynb.
Converted 0_template.ipynb.
Converted Untitled-1.ipynb.
Converted bot pseudocode.ipynb.
Converted bot_test.ipynb.
Converted classifying_glovo_images.ipynb.
Converted foods_prompted_tosql.ipynb.
Converted ideas.ipynb.
Converted inference.ipynb.
Converted multiple3105.ipynb.
Converted multiple_3005.ipynb.
No export destination, ignored:
#export
def search_image(url=None,head = 1):
    image_clip = requests.post(f'https://guru.skynet.center/image2vector/?url={url}').json()
    results = client.search(collection_name=collection_name,query_vector=image_clip,top=head)
    image_clip = torch.Tensor(image_clip)
    df =